In [1]:
from itertools import product
import numpy as np

In [2]:
from criterias import overalloc, time_conflicts, undersupport, unwilling, unpreffered, testfiles
from criterias import tas_n, sect_n
from agents import mutating_agent, min_agent, min_agent_ran, add_preferred_courses
from profiler import Profiler, profile
from evo import Evo

In [3]:
global tas_n
global sect_n

In [6]:
# sect_n

In [7]:
@profile
def find_conflicts(assignment, conflicting_labs):
    conflicts = []

    # Iterate over each group of conflicting labs
    for conflict_group in conflicting_labs:
        # Sum the assignments in the conflicting labs
        sub_matrix = assignment[:, conflict_group]  # Selecting columns for the labs
        conflicts_sum = np.sum(sub_matrix, axis=1)  # Sum along the columns now to get sum per TA

        # Identify TAs with conflicts
        # Conflict occurs if a TA is assigned to more than one lab in the group
        conflict_indices = np.where(conflicts_sum > 1)[0]  # TAs indices with conflicts

        # Append conflicts for each TA
        for ta_index in conflict_indices:
            # Find specific labs where the TA is assigned within this conflict group
            assigned_labs = [lab for lab in conflict_group if assignment[ta_index, lab] == 1]
            conflicts.append([(ta_index, lab) for lab in assigned_labs])

    return conflicts

@profile
def generate_variants(base_matrix, conflicts):
    # Initialize the list of all variant matrices
    variants = []
    
    # Create a list of lists, where each sublist contains all possible configurations for each conflict group
    all_combinations = []
    
    for conflict_group in conflicts:
        group_combinations = []
        for coord in conflict_group:
            # Create a new copy of the base matrix for each possibility
            temp_matrix = base_matrix.copy()
            # Set all conflicted coordinates to 0
            for c in conflict_group:
                temp_matrix[c[0], c[1]] = 0
            # Set the current coordinate to 1
            temp_matrix[coord[0], coord[1]] = 1
            group_combinations.append(temp_matrix)
        all_combinations.append(group_combinations)
    
    # Generate the product of all combinations across all conflict groups
    for combination in product(*all_combinations):
        # Combine matrices from each group into a single matrix
        combined_matrix = sum(combination)
        # Using 'minimum' to avoid overlapping 1's going beyond 1, if any
        combined_matrix = np.minimum(combined_matrix, 1)
        variants.append(combined_matrix)

    return variants

@ profile
def create_prefs():
    '''
    create all possible combinations of preferene matrixes where everyone is willing
    '''
    
    # Create solution where everyone has their preferences
    pref_sol = tas_n[:, 2:]
    # Replace all 1s and 2s with 0s; Replace all 3s with 1s
    pref_sol[(pref_sol == 1) | (pref_sol == 2)] = 0
    pref_sol[pref_sol == 3] = 1
    
    # Find the Labs with time conflicts:
    conflicting_labs = {}
    for lab_id, time_id in sect_n[:, [0,1]]:
        if time_id in conflicting_labs:
            conflicting_labs[time_id].append(lab_id)
        else:
            conflicting_labs[time_id] = [lab_id]
    conflicting_labs_list = list(conflicting_labs.values())
    # Find list of lists of conflicts
    conflicts = find_conflicts(pref_sol, conflicting_labs_list)
    
    # Create all solutions where no one has a time conflict:
    return generate_variants(pref_sol, conflicts)

In [8]:
'''
create all possible combinations of preferene matrixes where everyone is willing
'''

# Create solution where everyone has their preferences
pref_sol = tas_n[:, 2:]
# Replace all 1s and 2s with 0s; Replace all 3s with 1s
pref_sol[(pref_sol == 1) | (pref_sol == 2)] = 0
pref_sol[pref_sol == 3] = 1

In [9]:
# Find the Labs with time conflicts:
conflicting_labs = {}
for lab_id, time_id in sect_n[:, [0,1]]:
    if time_id in conflicting_labs:
        conflicting_labs[time_id].append(lab_id)
    else:
        conflicting_labs[time_id] = [lab_id]

In [10]:
conflicting_labs_list = list(conflicting_labs.values())
# Find list of lists of conflicts
conflicts = find_conflicts(pref_sol, conflicting_labs_list)

In [11]:
# Create all solutions where no one has a time conflict:
generate_variants(pref_sol, conflicts)

Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x0000021FBAA4A750>>
Traceback (most recent call last):
  File "C:\Users\ivnsa\anaconda3\envs\ds35\Lib\site-packages\ipykernel\ipkernel.py", line 770, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(

KeyboardInterrupt: 


KeyboardInterrupt: 

In [12]:
len(variants)

NameError: name 'variants' is not defined

In [60]:
pref_sol = tas_n[:, 2:]
pref_sol

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [17]:
sect

,section,daytime,min_ta
0,0,0,3
1,1,1,3
2,2,1,2
3,3,1,2
4,4,2,3
5,5,2,3
6,6,3,2
7,7,3,2
8,8,3,3
9,9,4,3


In [33]:
# Example usage
base_matrix = np.zeros((3, 3))  # Example base matrix
base_matrix[0, 1] = 1
base_matrix[1, 1] = 1
base_matrix[2, 1] = 1

conflicts = [[(0, 1), (1, 1)], [(1, 1), (2, 1)]]

variants = generate_variants(base_matrix, conflicts)

# Print variants
for i, var in enumerate(variants):
    print(f"Variant {i+1}:\n{var}\n")

Variant 1:
[[0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]]

Variant 2:
[[0. 1. 0.]
 [0. 0. 0.]
 [0. 1. 0.]]

Variant 3:
[[0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]]

Variant 4:
[[0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]]


In [22]:
sect_n[:, [0,1]]

array([[ 0,  0],
       [ 1,  1],
       [ 2,  1],
       [ 3,  1],
       [ 4,  2],
       [ 5,  2],
       [ 6,  3],
       [ 7,  3],
       [ 8,  3],
       [ 9,  4],
       [10,  5],
       [11,  5],
       [12,  0],
       [13,  0],
       [14,  0],
       [15,  6],
       [16,  6]], dtype=int64)

In [13]:
tas_n[:, 2:]

array([[0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0],
       [0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
       [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
       [0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0],
       [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,